In [7]:
def crawl_detail_info(detail_url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(detail_url)
#     time.sleep(1)
    html_txt = driver.page_source
    soup = BeautifulSoup(html_txt, 'html.parser')

   
    info_dict = {}
    
    try:
        name_tag = soup.select_one("#bobaeConent > div.component.page-detail > div.container-detail > div.wrap-detail-spot.js-object-fit > div.group-info > div.info-price.box.mode-basic > div.title-area > h3")
        info_dict["차종"] = name_tag.text.strip()
    except:
        info_dict["차종"] = None

    
    try:
        price_tag = soup.select_one("#bobaeConent > div.component.page-detail > div.container-detail > div.wrap-detail-spot.js-object-fit > div.group-info > div.info-price.box.mode-basic > div.price-area > span")
        info_dict["가격"] = price_tag.text.strip()
    except:
        info_dict["가격"] = None
    try:
        tbody_tag = soup.select_one("#bobaeConent > div.component.page-detail > div.container-detail > div.detail-section.mode-half > div:nth-child(1) > div.info-basic > div > table > tbody")
    except:
        pass

    # 각 tr 태그에 대해 반복하여 정보를 추출하여 딕셔너리에 저장하고, 리스트에 추가
    for tr_tag in tbody_tag.find_all('tr'):
        th_tags = tr_tag.find_all('th')
        td_tags = tr_tag.find_all('td')
        
        for th_tag, td_tag in zip(th_tags, td_tags):
            # 각 th 태그와 td 태그의 텍스트를 키와 값으로 저장
            key = th_tag.text.strip()
            value = td_tag.text.strip()
            if key not in ["배기량","연식","주행거리","변속기","색상","연료"]:
                break
            info_dict[key] = value        
#     - 소유자변경유무
#     - 전손유무
#     - 침수유무
        flt_tags = soup.select("#bobaeConent > div.component.page-detail > div.container-detail > div:nth-child(5) > div > div.info-insurance.mode-personal > div.insurance-cont > dl.flt > dd")
        if len(flt_tags) >= 2:
            info_dict["소유자 이전 횟수"] = flt_tags[0].text.strip().split("/")[-1]
            info_dict["전손유무"] = flt_tags[1].text.strip().split("/")[0]
            info_dict["침수유무"] = flt_tags[1].text.strip().split("/")[2] if len(flt_tags[1].text.strip().split("/")) > 2 else None
# - 옵션
#     - 썬루프
#     - 내비게이션
#     - 스마트키
#     - 오토라이트
#     - 주차감지센서
#     - 가죽시트
#     - 열선시트
#     - 통풍시트
#     option_selector = ".detail-option-container"
    
#     option_containers = soup.select(option_selector)
    
    # 옵션들을 찾기 위한 선택자
    option_selector = ".option-list-container"
    
    # 옵션들을 모두 선택
    option_containers = soup.select(option_selector)
    try:
        for option_container in option_containers:
        # 대분류에서 모든 옵션을 선택
            options = option_container.select("span > label > button")

            # 각 옵션에 대해 순회하면서 필요한 정보 가져오기
            for option in options:
                # 버튼 태그에서 텍스트를 가져옴
                button_text = option.text.strip()

                # 버튼 텍스트가 리스트에 없는 경우 건너뜀
                if button_text not in ["선루프", "네비게이션(순정)","네비게이션(비순정)", "스마트키", "LED헤드램프","어댑티드헤드램프", "후방센서", "가죽시트", "열선시트(앞좌석)","열선시트(뒷자석)", "통풍시트"]:
                    continue

                # 버튼 태그의 부모 li 태그에서 checked 속성을 찾음
                checked_attribute = option.find_parent("li").find("input", {"checked": True})

                # checked 속성 존재하면 1
                if checked_attribute:
                    value = 1
                else:
                    value = 0

                # 대분류와 옵션 텍스트를 조합하여 키로, 값으로 설정하여 사전에 저장
                info_dict[button_text] = value
    except:
        pass
    
    return info_dict


In [8]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

base_url = "https://www.bobaedream.co.kr/cyber/CyberCar.php?gubun=K"

# 각 페이지마다의 상세 페이지 링크를 저장
detail_links = []


# 페이지 이동
for page_num in range(1, 2):  # 숫자조절하면서 페이지반복 1페이지부터 706페이지까지 반복해야함
    url = f"{base_url}&page={page_num}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 각 페이지에서 상세 페이지 링크 추출
    car_elements = soup.select('#listCont > div.wrap-thumb-list > ul > li > div > div.mode-cell.title > p.tit.ellipsis > a')
    for car_element in car_elements:
        detail_link = car_element['href']
        detail_links.append(detail_link)
    
    print(page_num)
# 상세 정보를 저장할 리스트 초기화
car_info_list = []

# print(detail_links)

# time.sleep(1)

# 각 상세 페이지에 접속하여 정보 크롤링
for detail_link in detail_links:
    detail_link = f"https://www.bobaedream.co.kr{detail_link}" 
    detail_info = crawl_detail_info(detail_link)
    car_info_list.append(detail_info)
#     print(car_info_list)

1


KeyboardInterrupt: 

In [5]:
import csv


fields = ['차종','가격', '연식', '배기량', '주행거리', '색상', '변속기', '연료', '선루프', 'LED헤드램프', '어댑티드헤드램프', '가죽시트', '열선시트(앞좌석)', '통풍시트', '후방센서', '스마트키', '네비게이션(순정)', '네비게이션(비순정)','전손유무','소유자 이전 횟수','침수유무']

filename = 'car_data.csv'
with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:  
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    
    writer.writeheader()
    
    for car in car_info_list:
        writer.writerow(car)

In [9]:
# 실행중간에 멈추고 확인할때
for car_info in car_info_list:
    print(car_info)

{'차종': 'GM대우 레스타  캠핑카 -', '가격': '4,680만원', '연식': '2013.07', '배기량': '3,760 cc ()', '주행거리': '161,867 km', '색상': '흰색투톤', '변속기': '수동', '연료': '디젤', '선루프': 0, 'LED헤드램프': 0, '어댑티드헤드램프': 0, '가죽시트': 1, '열선시트(앞좌석)': 0, '통풍시트': 0, '후방센서': 1, '스마트키': 0, '네비게이션(순정)': 0, '네비게이션(비순정)': 1}
{'차종': '현대 그랜저IG  3.0 LPi  프리미엄 스페셜', '가격': '2,190만원', '연식': '2018.09\xa0(19년형)', '배기량': '2,999 cc (235마력)', '주행거리': '117,609 km', '색상': '흰색', '변속기': '자동', '연료': 'LPG', '선루프': 0, 'LED헤드램프': 1, '어댑티드헤드램프': 0, '가죽시트': 1, '열선시트(앞좌석)': 1, '통풍시트': 1, '후방센서': 1, '스마트키': 1, '네비게이션(순정)': 1, '네비게이션(비순정)': 0}
{'차종': '제네시스 GV80  3.0 디젤 5인승', '가격': '5,990만원', '연식': '2021.09', '배기량': '2,996 cc (278마력)', '주행거리': '22,097 km', '색상': '진회색', '변속기': '자동', '연료': '디젤', '선루프': 0, 'LED헤드램프': 1, '어댑티드헤드램프': 0, '가죽시트': 1, '열선시트(앞좌석)': 1, '통풍시트': 1, '후방센서': 1, '스마트키': 1, '네비게이션(순정)': 1, '네비게이션(비순정)': 0}
{'차종': '기아 카니발 4세대  3.5 가솔린 7인승 시그니처', '가격': '4,350만원', '연식': '2022.07\xa0(23년형)', '배기량': '3,470 cc (294마력)', '주행거리': '22,635 km', '색상': '흰색

In [6]:
for detail_link in detail_links:
    detail_link = f"https://www.bobaedream.co.kr{detail_link}" 
    detail_info = crawl_detail_info(detail_link)
    car_info_list.append(detail_info)

KeyboardInterrupt: 